In [7]:
import sys
!{sys.executable} -m pip install -q anthropic google-generativeai openai

In [4]:
from anthropic import Anthropic       #pip install anthropic
import google.generativeai as genai   #pip install google-generativeai
from openai import OpenAI             #pip install openai
import time

c:\Users\Lenovo\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_16360\2823137590.py:2: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  import google.generativeai as genai   #pip install google-generativeai


In [6]:
import os
from dotenv import load_dotenv

# Load keys from .env file
load_dotenv()

ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Initialize clients
anthropic_client = None
gemini_model = None
openai_client = None

# 1. Setup Claude
if ANTHROPIC_API_KEY:
    try:
        anthropic_client = Anthropic(api_key=ANTHROPIC_API_KEY)
        print("✅ Claude ready")
    except Exception as e:
        print(f"❌ Claude failed: {e}")

# 2. Setup Gemini
if GOOGLE_API_KEY:
    try:
        genai.configure(api_key=GOOGLE_API_KEY)
        # Note: 'gemini-2.5-flash' might be a typo in your original file. 
        # Usually, it's 'gemini-1.5-flash' or 'gemini-1.5-pro'.
        gemini_model = genai.GenerativeModel('gemini-2.5-flash')
        print("✅ Gemini ready")
    except Exception as e:
        print(f"❌ Gemini failed: {e}")

# 3. Setup OpenAI
if OPENAI_API_KEY:
    try:
        openai_client = OpenAI(api_key=OPENAI_API_KEY)
        print("✅ GPT-4 ready")
    except Exception as e:
        print(f"❌ GPT-4 failed: {e}")

✅ Claude ready
✅ Gemini ready
✅ GPT-4 ready


In [7]:
def call_claude(prompt, max_tokens=1024):
    if not anthropic_client:
        return "[Claude unavailable]"
    try:
        msg = anthropic_client.messages.create(
            model="claude-3-5-sonnet-latest",
            max_tokens=max_tokens,
            messages=[{"role": "user", "content": prompt}]
        )
        return msg.content[0].text
    except Exception as e:
        return f"[Error: {e}]"

def call_gemini(prompt):
    if not gemini_model:
        return "[Gemini unavailable]"
    try:
        return gemini_model.generate_content(prompt).text
    except Exception as e:
        return f"[Error: {e}]"

def call_gpt(prompt, max_tokens=1024):
    if not openai_client:
        return "[GPT unavailable]"
    try:
        resp = openai_client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens
        )
        return resp.choices[0].message.content
    except Exception as e:
        return f"[Error: {e}]"

def compare(prompt, label=""):
    print(f"\n{'='*80}\n{label}\n{'='*80}\n")
    print(f"PROMPT:\n{'-'*80}\n{prompt}\n{'-'*80}\n")
    
    print("🔵 CLAUDE:")
    print(call_claude(prompt))
    time.sleep(1)
    
    print("\n🟢 GEMINI:")
    print(call_gemini(prompt))
    time.sleep(1)
    
    print("\n🟣 GPT-4:")
    print(call_gpt(prompt))

def compare_gemini_only(prompt, label=""):
    """Use only Gemini to demonstrate prompt quality"""
    print(f"\n{'='*80}\n{label}\n{'='*80}\n")
    print(f"PROMPT:\n{'-'*80}\n{prompt}\n{'-'*80}\n")
    
    print("\n🟢 GEMINI RESPONSE:")
    print(call_gemini(prompt))

***Defining Between Vague Prompt Vs Specific Instructions***

In [8]:
report = """Q3 2024: Revenue $45.2M (+23% YoY). SaaS: $32M (71%). 
Gross margin: 68%. OpEx: $28M (+18%). CAC: $1,200 (from $1,450). 
142 new enterprise deals. MRR: $10.2M (+19%). Churn: 2.1%. 
Challenges: EU regulatory delays, AI features delayed to Q1 2025."""

print("\n" + "="*80)
print("EXPERIMENT 1: VAGUE VS SPECIFIC INSTRUCTIONS")
print("="*80)

# Bad prompt
compare(
    f"Summarize this report:\n\n{report}",
    "❌ BAD: Vague request"
)

# Good prompt
compare(
    f"""### ROLE ###
You are a financial analyst for C-suite executives.

### TASK ###
Summarize this quarterly report with ONLY critical metrics and risks.

### FORMAT ###
- Exactly 4 bullet points
- Max 25 words each
- Start each with a number/percentage
- Include 1 risk point

### REPORT ###
{report}""",
    "✅ GOOD: Structured with constraints"
)


EXPERIMENT 1: VAGUE VS SPECIFIC INSTRUCTIONS

❌ BAD: Vague request

PROMPT:
--------------------------------------------------------------------------------
Summarize this report:

Q3 2024: Revenue $45.2M (+23% YoY). SaaS: $32M (71%). 
Gross margin: 68%. OpEx: $28M (+18%). CAC: $1,200 (from $1,450). 
142 new enterprise deals. MRR: $10.2M (+19%). Churn: 2.1%. 
Challenges: EU regulatory delays, AI features delayed to Q1 2025.
--------------------------------------------------------------------------------

🔵 CLAUDE:
[Error: Error code: 401 - {'type': 'error', 'error': {'type': 'authentication_error', 'message': 'invalid x-api-key'}, 'request_id': 'req_011CWCepFatYxtkmNeMC71f1'}]

🟢 GEMINI:
For Q3 2024, the company reported strong financial performance with total revenue reaching $45.2 million, a 23% increase year-over-year. SaaS revenue accounted for $32 million (71% of total), contributing to an MRR of $10.2 million, up 19%.

Gross margin stood at 68%, while operating expenses increase

***Analysing Code Generation Quality With And Without Specifications***

In [10]:
print("\n" + "="*80)
print("EXPERIMENT 2: CODE GENERATION QUALITY (Gemini Only)")
print("Focus: How prompt specificity affects code quality")
print("="*80)

compare_gemini_only(
    "Write a function to grade students based on their scores.",
    "❌ BAD: No logic or format specified"
)

compare_gemini_only(
    """Write a Python function:

NAME: get_student_grades

INPUT: A dictionary where keys are student names (str) and values are lists of test scores (List[int]).

REQUIREMENTS:
1. Calculate the average score for each student.
2. Assign a letter grade based on the average: 
   - 90+ : 'A'
   - 80-89: 'B'
   - 70-79: 'C'
   - Below 70: 'F'
3. Return a new dictionary: {student_name: grade_letter}.
4. If a student has no scores, assign them 'Incomplete'.
5. Include type hints, a docstring, and 2 test cases (one normal, one with an empty list).
6. The code must be ready to copy-paste and run immediately.
7. Output only the code block. Remove all markdown triple backticks.

OUTPUT: Complete runnable code only""",
    "✅ GOOD: Specific Logic and Handling Edge Cases"
)


EXPERIMENT 2: CODE GENERATION QUALITY (Gemini Only)
Focus: How prompt specificity affects code quality

❌ BAD: No logic or format specified

PROMPT:
--------------------------------------------------------------------------------
Write a function to grade students based on their scores.
--------------------------------------------------------------------------------


🟢 GEMINI RESPONSE:
Grading students based on scores typically involves defining ranges for different letter grades (A, B, C, D, F). Here's a Python function that does this, along with explanations and examples.

### Function to Grade a Single Student

This function takes a single score and returns the corresponding letter grade.

```python
def grade_student(score: float) -> str:
    """
    Grades a student based on their numerical score using a standard A-F scale.

    Args:
        score (float): The student's numerical score (expected between 0 and 100).

    Returns:
        str: The letter grade (A, B, C, D, F) or "

***How Zero-Shot And Few-Shot Effect Output Format***

In [11]:
print("\n" + "="*80)
print("EXPERIMENT 3: ZERO-SHOT VS FEW-SHOT LEARNING (Gemini Only)")
print("Focus: Impact of examples on classification accuracy")
print("="*80)

compare_gemini_only(
    """Classify sentiment: Positive, Negative, or Neutral

Review: "The product arrived damaged, but customer service sent a replacement immediately."
Sentiment:""",
    "ZERO-SHOT: No examples"
)

compare_gemini_only(
    """Classify sentiment considering BOTH product and service:

EXAMPLES:
"Amazing quality!": Positive
"Broke after one day": Negative  
"Product okay, slow shipping": Neutral
"Defective but great replacement service": Neutral

Review: "The product arrived damaged, but customer service sent a replacement immediately."
Sentiment:""",
    "FEW-SHOT: With examples"
)



EXPERIMENT 3: ZERO-SHOT VS FEW-SHOT LEARNING (Gemini Only)
Focus: Impact of examples on classification accuracy

ZERO-SHOT: No examples

PROMPT:
--------------------------------------------------------------------------------
Classify sentiment: Positive, Negative, or Neutral

Review: "The product arrived damaged, but customer service sent a replacement immediately."
Sentiment:
--------------------------------------------------------------------------------


🟢 GEMINI RESPONSE:
Positive

FEW-SHOT: With examples

PROMPT:
--------------------------------------------------------------------------------
Classify sentiment considering BOTH product and service:

EXAMPLES:
"Amazing quality!": Positive
"Broke after one day": Negative  
"Product okay, slow shipping": Neutral
"Defective but great replacement service": Neutral

Review: "The product arrived damaged, but customer service sent a replacement immediately."
Sentiment:
-------------------------------------------------------------------

***How Chain-Of-Thought Effects A Prompt In Giving Outputs***

In [12]:
print("\n" + "="*80)
print("EXPERIMENT 4: CHAIN-OF-THOUGHT REASONING (Gemini Only)")
print("="*80)


math_problem = """A store has 156 laptops. Monday: sold 23% of inventory. 
Tuesday: received 85 new laptops. Wednesday: sold 42 laptops. 
How many laptops now?"""

compare(
    f"{math_problem}\n\nProvide only the final number.",
    "WITHOUT CoT : Direct answer requested"
)

compare(
    f"""{math_problem}

Let's solve step by step:
1. Calculate Monday's sales (23% of 156)
2. Remaining after Monday
3. Add Tuesday's shipment (+85)
4. Subtract Wednesday's sales (-42)
5. Final answer

Show calculations for each step.""",
    "WITH CoT"
)


EXPERIMENT 4: CHAIN-OF-THOUGHT REASONING (Gemini Only)

WITHOUT CoT : Direct answer requested

PROMPT:
--------------------------------------------------------------------------------
A store has 156 laptops. Monday: sold 23% of inventory. 
Tuesday: received 85 new laptops. Wednesday: sold 42 laptops. 
How many laptops now?

Provide only the final number.
--------------------------------------------------------------------------------

🔵 CLAUDE:
[Error: Error code: 401 - {'type': 'error', 'error': {'type': 'authentication_error', 'message': 'invalid x-api-key'}, 'request_id': 'req_011CWCgU6XMrGtZTAbxaKxAK'}]

🟢 GEMINI:
164

🟣 GPT-4:
[Error: Error code: 401 - {'error': {'message': 'Incorrect API key provided: your_ope********here. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}]

WITH CoT

PROMPT:
--------------------------------------------------------------------------------
A sto

***How Role Playing(Persona) Helps in Getting a Desired Output***

In [13]:
print("\n" + "="*80)
print("EXPERIMENT 5: ROLE/PERSONA IMPACT")
print("="*80)

compare_gemini_only(
    "Explain machine learning in 3 sentences.",
    "NO ROLE"
)

compare_gemini_only(
    """You are a CS professor teaching freshmen with no coding background.

Explain machine learning in exactly 3 sentences:
1. What it is (use everyday analogy)
2. How it works (simple mechanism)  
3. Real-world example

Avoid jargon. Conversational tone.""",
    "WITH PROFESSOR ROLE"
)

compare_gemini_only(
    """You are explaining to a curious 8-year-old.

Explain machine learning using:
- Simple words only
- Comparison to toys/pets/games
- Max 3 short sentences
- Fun, enthusiastic tone""",
    "WITH CHILD-FRIENDLY ROLE"
)


EXPERIMENT 5: ROLE/PERSONA IMPACT

NO ROLE

PROMPT:
--------------------------------------------------------------------------------
Explain machine learning in 3 sentences.
--------------------------------------------------------------------------------


🟢 GEMINI RESPONSE:
Machine learning is a field of artificial intelligence where computers learn from data to identify patterns and make decisions or predictions without being explicitly programmed. Instead of following fixed instructions, algorithms are trained on large datasets to recognize relationships and features within the information. This allows systems to perform tasks like image recognition, language translation, or recommendation generation by applying the knowledge gained from their training.

WITH PROFESSOR ROLE

PROMPT:
--------------------------------------------------------------------------------
You are a CS professor teaching freshmen with no coding background.

Explain machine learning in exactly 3 sentences:
1. 

***How can we Control The Output Format***

In [14]:
print("\n" + "="*80)
print("EXPERIMENT 6: OUTPUT FORMAT CONTROL")
print("="*80)

text = """Sarah Johnson, 32, Senior Developer at TechCorp. 
Email: sarah.j@techcorp.com, Phone: +1-555-0123. 
Started 2019. Skills: Python, JavaScript. Reports to Mike Chen."""

compare_gemini_only(
    f"Extract information from:\n{text}",
    "NO FORMAT SPECIFIED"
)

compare_gemini_only(
    f"""Extract info and output ONLY valid JSON with this structure:

{{
  "name": "string",
  "age": number,
  "company": "string",
  "position": "string",
  "contact": {{"email": "string", "phone": "string"}},
  "employment": {{
    "start_year": number,
    "skills": ["string"],
    "manager": "string"
  }}
}}

TEXT: {text}

Return ONLY JSON, no markdown or extra text.""",
    "WITH JSON FORMAT"
)


EXPERIMENT 6: OUTPUT FORMAT CONTROL

NO FORMAT SPECIFIED

PROMPT:
--------------------------------------------------------------------------------
Extract information from:
Sarah Johnson, 32, Senior Developer at TechCorp. 
Email: sarah.j@techcorp.com, Phone: +1-555-0123. 
Started 2019. Skills: Python, JavaScript. Reports to Mike Chen.
--------------------------------------------------------------------------------


🟢 GEMINI RESPONSE:
Here's the extracted information:

*   **Name:** Sarah Johnson
*   **Age:** 32
*   **Title/Position:** Senior Developer
*   **Company:** TechCorp
*   **Email:** sarah.j@techcorp.com
*   **Phone:** +1-555-0123
*   **Start Date (at Company):** 2019
*   **Skills:** Python, JavaScript
*   **Manager/Reports to:** Mike Chen

WITH JSON FORMAT

PROMPT:
--------------------------------------------------------------------------------
Extract info and output ONLY valid JSON with this structure:

{
  "name": "string",
  "age": number,
  "company": "string",
  "posit

***Resolving the Problem Of Ambiguity*** 

In [15]:
print("\n" + "="*80)
print("EXPERIMENT 7: AMBIGUITY RESOLUTION")
print("="*80)

compare_gemini_only(
    "Write about Python.",
    "AMBIGUOUS"
)

compare_gemini_only(
    """CONTEXT: I'm a Java developer learning Python programming.

TASK: Compare Java and Python syntax with 3 key differences.

FORMAT (for each):
- Feature name
- Java code example
- Python code example  
- 1-sentence explanation

CONSTRAINTS:
- Syntax only (not performance)
- Simple examples (variables, loops, functions)
- Max 5 lines of code each""",
    "DISAMBIGUATED"
)


EXPERIMENT 7: AMBIGUITY RESOLUTION

AMBIGUOUS

PROMPT:
--------------------------------------------------------------------------------
Write about Python.
--------------------------------------------------------------------------------


🟢 GEMINI RESPONSE:
[Error: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash
Please retry in 50.893608052s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
  